### **Script base: Spazzata di voltaggio**

Questo script esegue una spazzata lineare dal voltaggio `V0` al voltaggio `V1` con  `nV` punti. Per ogni valore l'$\texttt{ADC}$ esegue una misura con i seguenti parametri di campionamento:

* frequenza di sampling `fs`
* numero di acquisizioni `npt`

I valori risultanti di $\texttt{Ch1}$ e $\texttt{Ch2}$ vengono mediati e salvati.

In [ ]:
### Per task 2

import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time

V0 = 0
V1 = 5
nV = 101
npt = 101
fs  = 1e6

Vv = np.linspace(V0,V1,nV)

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()
# ad2.vdd = 5
# ad2.vss = 0
# ad2.power(True)
#   3. Configurazione generatore funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.func = tdwf.funcDC
#wgen.w1.offs = V0
wgen.w2.func = tdwf.funcDC
wgen.w2.offs = 5    #configurazione per task 2
wgen.w2.sync = True
wgen.w1.start()
#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=fs
scope.npt=npt
scope.ch1.rng = 50
scope.ch2.rng = 50

# -[Ciclo di misura]-----------------------------------------------------------
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
fig.canvas.manager.set_window_title('Spazzata voltaggio')
ax2.yaxis.tick_right()
Ch1v = np.full((npt, 1), np.nan)
Ch2v = np.full((npt, 1), np.nan)

for ii, V in enumerate(Vv):
    wgen.w1.offs = V
    time.sleep(0.1)
    scope.sample()
    Ch1v[ii] = np.mean(scope.ch1.vals)
    Ch2v[ii] = np.mean(scope.ch2.vals)

    ax1.clear()    
    ax1.plot(Vv, Ch1v, ".", color="tab:orange", label="Ch1")
    ax1.plot(Vv, Ch2v, ".", color="tab:blue", label="Ch2")
    ax1.grid(True)
    ax1.set_xlabel("W1 [V]", fontsize=15)
    ax1.set_ylabel("Signals [V]", fontsize=15)
    ax1.legend()
    
    ax2.clear()    
    ax2.plot(Ch1v, Ch2v, ".", color="tab:orange")
    ax2.grid(True)
    ax2.set_xlabel("Ch1 [V]", fontsize=15)
    ax2.set_ylabel("Ch2 [V]", fontsize=15)
    ax2.yaxis.set_label_position('right')
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.001)

ad2.close()

data = np.column_stack((Vv, Ch2v))

save = False

if save:
    path = "/home/marco/Desktop/Uni_anno3/TD/Es_08/acquisizioni/"
    np.savetxt(path+"funzione_risposta_NOT.txt", data, header="W1 [V], Ch2 [V]", delimiter=",")

Dispositivo #1 [SN:210321B5D8FC, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.


In [ ]:
### Per task 3

import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time

path1 = "/home/marco/Desktop/Uni_anno3/TD/Es_08/acquisizioni/"
path2 = "/home/marco/Desktop/Uni_anno3/TD/Es_08/logbook/"
path3 = "/home/marco/Desktop/Uni_anno3/TD/Es_08/presentazione/"

V0 = 0
V1 = 5
nV = 101
npt = 101
fs  = 1e6

Vv = np.linspace(V0,V1,nV)

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()
# ad2.vdd = 5
# ad2.vss = -5
# ad2.power(True)
#   3. Configurazione generatore funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.func = tdwf.funcDC
wgen.w1.offs = 0
wgen.w2.func = tdwf.funcDC
#wgen.w2.offs = 5    #configurazione per task 2
#wgen.w2.sync = True
wgen.w1.sync()    # configure master/slave (W1 -> W2)
wgen.w1.start()
wgen.w2.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=fs
scope.npt=npt
scope.ch1.rng = 50
scope.ch2.rng = 50

# -[Ciclo di misura]-----------------------------------------------------------

# Da modificare in modo da selezionare l'intervallo di valori di V_GS in cui V_DS passa da 5 a 0 V
W10 = 1
W11 = 2
nV1 = 7
Vv1 = np.linspace(W10,W11,nV1)

V_GS, V_DS = np.loadtxt(path1+"funzione_risposta_NOT.txt", delimiter=",", unpack=True)

fig1 = plt.figure(figsize=(10,6), dpi = 100, layout='constrained')
ax3, ax4 = fig1.subplots(2,1)

ax3.set_title("Funzione di risposta V_DS vs V_GS", fontsize=18)
ax3.plot(V_GS, V_DS, ".", color="tab:blue", label="Funzione di risposta NOT")
ax3.set_xlabel("V_GS [V]")
ax3.set_ylabel("V_DS [V]")
ax3.grid(True)
ax3.legend()

ax4.set_title("Caratteristica I_D vs V_DS", fontsize=18)
ax4.set_xlabel("V_DS [V]")
ax4.set_ylabel("I_D [A]")
ax4.grid(True)

for i in range(nV1):
    wgen.w1.offs = Vv1[i]
    #time.sleep(0.1)

    #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    #fig.canvas.manager.set_window_title(f'Spazzata voltaggio V_GS={Vv1[i]:.2f} V')

    #ax2.yaxis.tick_right()

    # Use 1-D arrays (not (npt,1))
    Ch1v = np.full(npt, np.nan)
    Ch2v = np.full(npt, np.nan)

    # Optionally create line objects for faster updates:
    #l1, = ax1.plot(Vv, Ch1v, ".", label="Ch1")
    #l2, = ax1.plot(Vv, Ch2v, ".", label="Ch2")
    #ax1.grid(True)
    #ax1.set_xlabel("W1 [V]", fontsize=15)
    #ax1.set_ylabel("Signals [V]", fontsize=15)
    #ax1.legend()

    #l3, = ax2.plot(Ch1v, Ch2v, ".", label="IV")
    #ax2.grid(True)
    #ax2.set_xlabel("Ch1 [V]", fontsize=15)
    #ax2.set_ylabel("Ch2 [V]", fontsize=15)
    #ax2.yaxis.set_label_position('right')
    #ax2.legend()

    #plt.tight_layout()

    for ii, V in enumerate(Vv):
        print(ii)
        wgen.w2.offs = V
        time.sleep(0.01)
        scope.sample()
        Ch1v[ii] = np.mean(scope.ch1.vals)
        Ch2v[ii] = np.mean(scope.ch2.vals)

        #Per il plot mi aspetto di visualizzare la caduta di potenziale ai capi della resistenza di pull up, quindi una retta, posso anche commentarlo se tutto funziona
        # update the plotted data (fast) instead of re-creating plots
        # update only valid values
        valid = ii + 1

        #l1.set_ydata(Ch1v[:valid])
        #l2.set_ydata(Ch2v[:valid])
        #l3.set_xdata(Ch1v[:valid])
        #l3.set_ydata(Ch2v[:valid])

        #ax1.relim()
        #ax1.autoscale_view()
        #ax2.relim()
        #ax2.autoscale_view()

        #fig.canvas.draw_idle()
        #plt.pause(0.001)

    # Plot caratteristica
    if i == 0:
        retta_carico = -Ch2v/1e3 + 5/1e3  #resistenza di carico da 1kOhm
        ax4.plot(Ch2v, retta_carico, "--", label="Retta di carico 1k$\Omega$")

    I_D = (Ch1v - Ch2v)/1e3  #resistenza di carico da 1kOhm

    ax4.plot(Ch2v, I_D, "-", label=f"I_D vs V_DS per V_GS = {Vv1[i]:.2f} V")

ax4.legend()

savefig = False

if savefig:
    plt.savefig(path2+"funz_risposta_caratteristica_ID_VDS_NOT.png", dpi=300)
    plt.savefig(path2+"PortaNOT.pdf", dpi=300)

    ax4.set_ylabel('I_D [A]', fontsize='18')
    ax4.set_xlabel("V_DS [V]", fontsize='18')
    ax3.set_xlabel('V_GS [V]', fontsize='18')
    ax3.set_ylabel('V_DS [V]', fontsize='18')
    ax3.tick_params(axis='x', labelsize=16)
    ax3.tick_params(axis='y', labelsize=16)
    ax4.tick_params(axis='x', labelsize=16)
    ax4.tick_params(axis='y', labelsize=16)
    ax3.legend(fontsize='16')
    #ax4.legend(fontsize='16')

    plt.savefig(path3 + "funz_risposta_caratteristica_ID_VDS_NOT.png", dpi=300)


plt.show()

ad2.close()

Dispositivo #1 [SN:210321B5D8FC, hdwf=1] connesso!
Configurazione #1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19